In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [16]:
h2o.init()
h2o.no_progress()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,28 mins 12 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_thinker_e4tyfa
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.938 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [3]:
data_df = pd.read_csv('../data/ieee-fraud-detection/train_transaction.csv')
data_df

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
feature = data_df.drop('isFraud', axis=1)
feature

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,15811047,49.00,W,6550,NaN,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
label = data_df['isFraud']
label

0         0
1         0
2         0
3         0
4         0
         ..
590535    0
590536    0
590537    0
590538    0
590539    0
Name: isFraud, Length: 590540, dtype: int64

In [20]:
y = 'isFraud'
X = list(data_df.columns)
# X.remove(y)

train, valid = train_test_split(
    data_df,
    test_size=0.2,
    random_state=42
)

In [24]:
h2o_train = h2o.H2OFrame(train)
h2o_valid = h2o.H2OFrame(valid)

In [26]:
y = 'isFraud'
h2o_train[y] = h2o_train[y].asfactor()
h2o_valid[y] = h2o_valid[y].asfactor()

In [27]:
aml = H2OAutoML(
    max_runtime_secs=120,
)
aml.train(
    x=X, 
    y=y, 
    training_frame=h2o_train, 
    leaderboard_frame=h2o_valid
)


21:31:47.374: AutoML: XGBoost is not available; skipping it.

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_2_AutoML_1_20220612_213144

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.015686776326441703
RMSE: 0.1252468615432806
LogLoss: 0.06756196128638868
Null degrees of freedom: 9997
Residual degrees of freedom: 9995
Null deviance: 3067.546752549424
Residual deviance: 1350.9689778826278
AIC: 1356.9689778826278
AUC: 0.9430206250465565
AUCPR: 0.7301806979802264
Gini: 0.886041250093113

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3062170287907319: 


,,0,1,Error,Rate
0,0,9603.0,40.0,0.0041,(40.0/9643.0)
1,1,142.0,213.0,0.4,(142.0/355.0)
2,Total,9745.0,253.0,0.0182,(182.0/9998.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.306217,0.700658,132.0
1,max f2,0.130881,0.669765,197.0
2,max f0point5,0.671684,0.807068,70.0
3,max accuracy,0.306217,0.981796,132.0
4,max precision,0.999015,1.000000,0.0
5,max recall,0.004121,1.000000,385.0
6,max specificity,0.999015,1.000000,0.0
7,max absolute_mcc,0.306217,0.702114,132.0
8,max min_per_class_accuracy,0.025257,0.859155,307.0
9,max mean_per_class_accuracy,0.049462,0.869823,265.0



Gains/Lift Table: Avg response rate:  3.55 %, avg score:  3.62 %


...

,,0,1,Error,Rate
0,0,45462.0,312.0,0.0068,(312.0/45774.0)
1,1,816.0,838.0,0.4933,(816.0/1654.0)
2,Total,46278.0,1150.0,0.0238,(1128.0/47428.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.293573,0.597718,165.0
1,max f2,0.103768,0.582753,247.0
2,max f0point5,0.647197,0.731164,88.0
3,max accuracy,0.582062,0.977946,98.0
4,max precision,0.998991,1.000000,0.0
5,max recall,0.001994,1.000000,397.0
6,max specificity,0.998991,1.000000,0.0
7,max absolute_mcc,0.516578,0.602284,111.0
8,max min_per_class_accuracy,0.020358,0.814994,335.0
9,max mean_per_class_accuracy,0.031204,0.828169,314.0



Gains/Lift Table: Avg response rate:  3.49 %, avg score:  3.42 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010015,0.905269,28.131417,28.131417,0.981053,0.974269,0.981053,0.974269,0.281741,0.281741,2713.141730,2713.141730,0.281545
1,2,0.020009,0.388420,17.906581,23.024386,0.624473,0.643245,0.802950,0.808931,0.178960,0.460701,1690.658116,2202.438639,0.456616
2,3,0.030003,0.206221,8.892795,18.317166,0.310127,0.285225,0.638791,0.634485,0.088875,0.549577,789.279537,1731.716633,0.538348
3,4,0.040019,0.131421,4.286117,14.805708,0.149474,0.163126,0.516333,0.516521,0.042926,0.592503,328.611723,1380.570778,0.572448
4,5,0.050013,0.098415,3.690208,12.584482,0.128692,0.112900,0.438870,0.435865,0.036880,0.629383,269.020760,1158.448220,0.600306
5,6,0.100004,0.041377,1.838279,7.212513,0.064108,0.061747,0.251529,0.248845,0.091898,0.721282,83.827864,621.251327,0.643727
6,7,0.150017,0.027289,1.148440,5.190871,0.040051,0.033245,0.181026,0.176969,0.057437,0.778718,14.843978,419.087134,0.651419
7,8,0.200008,0.020648,0.689354,4.065729,0.024040,0.023711,0.141788,0.138662,0.034462,0.813180,-31.064551,306.572940,0.635328
8,9,0.300013,0.013833,0.495747,2.875735,0.017289,0.016779,0.100288,0.098035,0.049577,0.862757,-50.425307,187.573524,0.583079
9,10,0.399996,0.010779,0.380959,2.252140,0.013286,0.012080,0.078541,0.076549,0.038089,0.900846,-61.904094,125.213982,0.518948


In [29]:
leaderboard = aml.leaderboard
performance = aml.leader.model_performance(h2o_valid)

model_id = aml.leader.model_id
accuracy = performance.accuracy()
precision = performance.precision()
recall = performance.recall()
F1 = performance.F1()
auc = performance.auc()
varimp = aml.leader.varimp()

print(model_id, accuracy, precision, recall, F1, auc, varimp)
print(performance)

StackedEnsemble_BestOfFamily_2_AutoML_1_20220612_213144 [[0.46626568598449325, 0.9772242354455244]] [[0.9989422803400916, 1.0]] [[0.0013293602186737119, 1.0]] [[0.28712149444800794, 0.6015765454294012]] 0.9022185388794142 None

ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.019682640871268628
RMSE: 0.1402948355117487
LogLoss: 0.08458822688896882
Null degrees of freedom: 118107
Residual degrees of freedom: 118105
Null deviance: 36559.48011559042
Residual deviance: 19981.09260280466
AIC: 19987.09260280466
AUC: 0.9022185388794142
AUCPR: 0.6236982134056719
Gini: 0.8044370777588283

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.28712149444800794: 


,,0,1,Error,Rate
0,0,113052.0,814.0,0.0071,(814.0/113866.0)
1,1,2067.0,2175.0,0.4873,(2067.0/4242.0)
2,Total,115119.0,2989.0,0.0244,(2881.0/118108.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.287121,0.601577,176.0
1,max f2,0.102907,0.586712,256.0
2,max f0point5,0.627103,0.725353,93.0
3,max accuracy,0.466266,0.977224,129.0
4,max precision,0.998942,1.000000,0.0
5,max recall,0.001329,1.000000,399.0
6,max specificity,0.998942,1.000000,0.0
7,max absolute_mcc,0.435866,0.606984,136.0
8,max min_per_class_accuracy,0.021264,0.825790,338.0
9,max mean_per_class_accuracy,0.024611,0.830318,332.0



Gains/Lift Table: Avg response rate:  3.59 %, avg score:  3.48 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010008,0.900477,27.182975,27.182975,0.976311,0.974298,0.976311,0.974298,0.272041,0.272041,2618.297486,2618.297486,0.271796
1,2,0.020007,0.431660,18.836731,23.011619,0.676545,0.669614,0.826492,0.822020,0.188355,0.460396,1783.673087,2201.161889,0.456795
2,3,0.030006,0.217052,8.581439,18.202916,0.308213,0.302688,0.653781,0.648958,0.085809,0.546205,758.143935,1720.291628,0.535429
3,4,0.040006,0.135621,4.950830,14.890596,0.177815,0.171540,0.534815,0.529629,0.049505,0.595710,395.083039,1389.059598,0.576406
4,5,0.050005,0.099606,3.654184,12.643694,0.131245,0.115946,0.454114,0.446907,0.036539,0.632249,265.418434,1164.369416,0.603935
5,6,0.100002,0.042669,1.994477,7.319537,0.071634,0.063244,0.262891,0.255092,0.099717,0.731966,99.447739,631.953659,0.655508
6,7,0.150007,0.027168,1.027713,5.222144,0.036912,0.033755,0.187560,0.181309,0.051391,0.783357,2.771265,422.214357,0.656945
7,8,0.200003,0.020591,0.877004,4.135951,0.031499,0.023561,0.148548,0.141875,0.043847,0.827204,-12.299576,313.595071,0.650567
8,9,0.300005,0.013707,0.483254,2.918385,0.017357,0.016657,0.104818,0.100136,0.048326,0.875530,-51.674557,191.838528,0.596966
9,10,0.399998,0.010785,0.344201,2.274880,0.012362,0.012041,0.081705,0.078113,0.034418,0.909948,-65.579941,127.487997,0.528948
